# Scraping WodUp

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append('../src/')
import wodupcrawler
import os
from bokeh.models import Div, ColumnDataSource, Legend, HoverTool, CustomJS, Slider, Column, Row
from bokeh.io import output_notebook, show, output_file, save, reset_output
import fitetl, wodupcrawler, sleepetl
import glob
import pandas as pd
from datetime import datetime
from bokeh.plotting import figure
import htmltext
from plotutils import plot_ts

output_notebook()

/Users/hasannagib/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Loading BokehJS ...

In [24]:
df_sleep = sleepetl.read_sleep_plot_df()

In [25]:
df_sleep.head()

,date,start,end,awake,rem,light,deep,duration,time_asleep,start_hour,...,start_7d_avg,end_7d_avg,start_7d_std,end_7d_std,start_hour_prev_day,end_hour_prev_day,date_str,ts_str,start_time,end_time
0,2020-11-17,2020-11-16 22:20:00,2020-11-17 07:04:00,1.152833,1.746667,4.235667,1.598167,524.0,7.580500,22.33,...,22.330000,7.070,NaN,NaN,NaN,NaN,Tue Nov 17 2020,Tue Nov 17 2020,10:20 PM,07:04 AM
1,2020-11-16,2020-11-15 22:30:00,2020-11-16 08:01:00,2.512333,1.484667,3.901833,1.617833,571.0,7.004333,22.50,...,22.415000,7.545,0.120208,0.671751,22.33,7.07,Mon Nov 16 2020,Mon Nov 16 2020,10:30 PM,08:01 AM
2,2020-11-15,2020-11-14 21:36:00,2020-11-15 07:19:00,1.564333,1.982167,3.915833,2.254333,583.0,8.152333,21.60,...,22.143333,7.470,0.478156,0.492443,22.50,8.02,Sun Nov 15 2020,Sun Nov 15 2020,09:36 PM,07:19 AM
3,2020-11-14,2020-11-13 20:35:00,2020-11-14 05:15:00,2.183333,1.166667,4.016667,1.300000,520.0,6.483333,20.58,...,21.752500,6.915,0.873742,1.180579,21.60,7.32,Sat Nov 14 2020,Sat Nov 14 2020,08:35 PM,05:15 AM
4,2020-11-13,2020-11-12 21:32:00,2020-11-13 07:11:30,1.650000,1.533333,5.016667,1.450000,579.5,8.000000,21.53,...,21.708000,6.968,0.763197,1.029257,20.58,5.25,Fri Nov 13 2020,Fri Nov 13 2020,09:32 PM,07:11 AM


In [20]:
for i in ['awake', 'rem', 'light', 'deep']:    
    df_sleep.loc[df_sleep[i].isna(), i] = df_sleep[i].mean()
    df_sleep.loc[df_sleep[i]==0, i] = df_sleep[i].mean()

In [23]:
df_sleep[df_sleep.deep==0]

,date,start,end,awake,rem,light,deep,duration,time_asleep,start_hour,...,20hr,7day_avg,start_7d_avg,end_7d_avg,start_7d_std,end_7d_std,date_str,ts_str,start_time,end_time


In [34]:
plot_sleep_schedule, plot_sleep_schedule_cds = plot_ts(
    df_sleep.query('date >= "2020-01-01"'),
    plot_height=400,
    plot_width=800,
    alphas=[0.75],
    xvar='date',
    ys=['end_hour', 'start_hour'],
    hover_vars=['start_time', 'end_time'],
    hide_hovers=['start_hour', 'end_hour'],
    units=['hour'],
    #x_range=plot_sleep_stages.x_range,
    y_range=[2, 30],
    ylabel='24-Hour',
    title='Sleep schedule',
    styles=['b'],
    palette=['grey'], #'#154ba6', '#3f8dff', '#7ec4ff', '#e73360'
    bounded_bar_label='sleep',
    tools='box_select,xwheel_pan,pan,reset,box_zoom',
    active_scroll='xwheel_pan',
    show_plot=False
);

plot_sleep_schedule.line(x="date", y='5hr', color="white", line_dash="4 4", line_width=3, alpha=0.95, source=plot_sleep_schedule_cds)
plot_sleep_schedule.line(x="date", y='20hr', color="white", line_dash="4 4", line_width=3, alpha=0.95, source=plot_sleep_schedule_cds)
plot_sleep_schedule.line(x="date", y='start_7d_avg', color="grey", line_width=3, alpha=0.95, source=plot_sleep_schedule_cds)
plot_sleep_schedule.line(x="date", y='end_7d_avg', color="grey", line_width=3, alpha=0.95, source=plot_sleep_schedule_cds)


show(plot_sleep_schedule)

In [35]:
plot_sleep_scatter = figure(
    plot_height=500,
    plot_width=500,
    x_range=(15,30),
    y_range=(3,10),
    tools="lasso_select, box_select, reset",
    x_axis_label='Start hour',
    y_axis_label='Time asleep (hours)',
)

plot_sleep_scatter.add_layout(Legend(), 'below')
plot_sleep_scatter.circle(
    x='start_hour', 
    y='time_asleep', 
    size=10,
    alpha=0.5,
    color='#3f8dff', #154ba6', #, '#3f8dff', '#7ec4ff', '#e73360'],
    source=plot_sleep_schedule_cds
)

plot_sleep_scatter.add_tools(HoverTool(
        tooltips=[
            ("Duration", "@time_asleep"),
            ("7day avg", "@7day_avg"),
            ("Date", "@date_str"),
            ("Start", "@start_time"),
            ("End", "@end_time"),
        ]
    ))

regline_cds = ColumnDataSource(data={'x':[0, 30], 'y':[df_sleep['time_asleep'].mean(), df_sleep['time_asleep'].mean()]})
plot_sleep_scatter.line(x='x', y='y', color="#154ba6", line_width=5, alpha=0.6, source=s2)

s.selected.js_on_change(
    'indices', 
    CustomJS(
        args={'s':plot_sleep_schedule_cds, 's2':regline_cds},
        code="""
            
            const inds = s.selected.indices;
            const d = s.data;
            var ym = []
            var xm = []

            if (inds.length == 0) {
                return;
            }
            else if (inds.length == 1){
                s2.data['y'] = [d['time_asleep'][inds[0]], d['time_asleep'][inds[0]]]
            }
            else if (inds.length > 1){
                for (var i = 0; i < inds.length; i++) {
                    ym.push(d['time_asleep'][inds[i]])
                    xm.push(d['start_hour'][inds[i]])
                }

                soln = linearRegression(xm, ym)
                m = soln[0]
                b = soln[1]

                console.log(m)
                console.log(b)


                s2.data['y'] = [b, 30*m+b]
            }
            
        s.change.emit();
        s2.change.emit();
            
        """
    )
)

plot_sleep_scatter.js_on_event('reset', CustomJS(
        args={'s':plot_sleep_schedule_cds, 's2':regline_cds},
        code="""
        const d = s.data;
        var xm = [].slice.call(d['start_hour'])
        var ym = [].slice.call(d['time_asleep'])

        soln = linearRegression(xm, ym)
        m = soln[0]
        b = soln[1]
        
        s2.data['y'] = [b, 30*m+b]        
        s2.change.emit();
        """
))


In [36]:
dash = Row(plot_sleep_scatter, plot_sleep_schedule)

In [37]:
output_file('test.html')

In [39]:
show(dash)

In [40]:
save(dash, template=htmltext.bokeh_template)

'/Users/hasannagib/Desktop/projects/Wahoo-Tickrs-Logs/notebooks/test.html'

In [21]:
df_sleep.query('date == "2019-07-18"')

,date,start,end,awake,rem,light,deep,duration,time_asleep,start_hour,...,9hr,20hr,7day_avg,start_7d_avg,end_7d_avg,start_7d_std,end_7d_std,date_str,start_time,end_time
491,2019-07-18,2019-07-17 20:08:00,2019-07-18 04:44:30,NaN,NaN,NaN,NaN,516.5,NaN,20.13,...,9,20,7.325,22.128571,6.798571,1.867426,1.457446,Thu Jul 18 2019,08:08 PM,04:44 AM


In [55]:
df = pd.read_csv('/Users/hasannagib/Documents/s3stage/fitbit/sleep.csv')

In [47]:
for i, row in df.iterrows():
    if not row['time_asleep'].isna():
        #df.loc[i, 'time_asleep'] = 0.85*row['duration']
        print(row)

AttributeError: 'float' object has no attribute 'isna'

In [52]:
df.to_csv('/Users/hasannagib/Documents/s3stage/fitbit/sleep.csv', index=None)

In [54]:
df.loc[df['time_asleep'].isna(), "time_asleep"] = df.loc[df['time_asleep'].isna(), "duration"] * 0.75

Series([], Name: duration, dtype: float64)

In [66]:
df_sleep.query('date=="2020-06-16"')

,date,start,end,awake,rem,light,deep,duration,time_asleep,start_hour,...,9hr,20hr,7day_avg,start_7d_avg,end_7d_avg,start_7d_std,end_7d_std,date_str,start_time,end_time
155,2020-06-16,2020-06-15 20:31:00,2020-06-16 05:10:00,NaN,NaN,NaN,NaN,519.0,NaN,20.52,...,9,20,7.008333,22.128571,6.434286,1.360998,1.327188,Tue Jun 16 2020,08:31 PM,05:10 AM


In [70]:
df.query('date=="Tuesday, Jun 16, 2020"')

,date,start,end,awake,rem,light,deep,duration,time_asleep,start_hour,end_hour
155,"Tuesday, Jun 16, 2020",2020-06-15 20:31:00,2020-06-16 05:10:00,NaN,NaN,NaN,NaN,519.0,389.25,20.52,5.17


In [26]:
In [1]: import numpy as np

In [2]: import statsmodels.api as sm

In [3]: spector_data = sm.datasets.spector.load(as_pandas=False)

In [4]: spector_data.exog = sm.add_constant(spector_data.exog, prepend=False)

# Fit and summarize OLS model
In [5]: mod = sm.OLS(spector_data.endog, spector_data.exog)

In [6]: res = mod.fit()

In [7]: print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.416
Model:                            OLS   Adj. R-squared:                  0.353
Method:                 Least Squares   F-statistic:                     6.646
Date:                Wed, 18 Nov 2020   Prob (F-statistic):            0.00157
Time:                        15:01:55   Log-Likelihood:                -12.978
No. Observations:                  32   AIC:                             33.96
Df Residuals:                      28   BIC:                             39.82
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.4639      0.162      2.864      0.0

In [28]:
res.params

array([ 0.46385168,  0.01049512,  0.37855479, -1.49801712])

In [44]:
res.conf_int()[0][0]

0.13209913282368269

In [37]:
res.tvalues

array([ 2.86405365,  0.53868506,  2.72003545, -2.85941908])

In [41]:
res.HC3_se

array([0.16968332, 0.01953693, 0.16179398, 0.55430079])

In [42]:
res.pvalues

array([0.00784052, 0.59436148, 0.01108768, 0.00792932])